In [3]:
import pygame
import random
import math

# 초기 설정
pygame.init()
WIDTH, HEIGHT = 600, 300
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Kalman Filter 1D Visualization (Pygame)")
clock = pygame.time.Clock()

# 시뮬레이션 초기값 설정 (Turtle 예시와 동일)
pos_actual = -250.0
vel_actual = 1.0
pos_est = pos_actual
vel_est = 0.0
P = [[1.0, 0.0],
     [0.0, 1.0]]
accel_var = 0.1
measure_var = 5.0

running = True
while running:
    # --- 이벤트 처리 ---
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # --- 실제 물체 이동 및 측정 ---
    pos_actual += vel_actual
    if pos_actual > 250:
        pos_actual = 250
        vel_actual = -vel_actual
    elif pos_actual < -250:
        pos_actual = -250
        vel_actual = -vel_actual
    pos_meas = pos_actual + random.gauss(0, math.sqrt(measure_var))

    # --- 칼만 필터 예측 ---
    dt = 1.0
    pred_pos = pos_est + vel_est * dt
    pred_vel = vel_est
    Pxx, Pxv, Pvv = P[0][0], P[0][1], P[1][1]
    Pxx = Pxx + 2*dt*Pxv + (dt**2) * Pvv
    Pxv = Pxv + dt * Pvv
    Pvv = Pvv + accel_var
    P = [[Pxx, Pxv], [Pxv, Pvv]]

    # --- 칼만 필터 업데이트 ---
    y = pos_meas - pred_pos
    S = P[0][0] + measure_var
    Kx = P[0][0] / S
    Kv = P[0][1] / S
    pos_est = pred_pos + Kx * y
    vel_est = pred_vel + Kv * y
    Pxx_new = (1 - Kx) * P[0][0]
    Pxv_new = (1 - Kx) * P[0][1]
    Pvv_new = P[1][1] - Kv * P[0][1]
    P = [[Pxx_new, Pxv_new], [Pxv_new, Pvv_new]]

    # --- 화면 그리기 ---
    screen.fill((255, 255, 255))  # 흰색 배경
    # 좌표 변환: -250~250 범위를 윈도우 픽셀 좌표로 (중앙 300, 범위±250을 ±250 픽셀로 사용)
    center_x = WIDTH // 2
    center_y = HEIGHT // 2
    # 실제 위치 (파란색 원)
    actual_px = int(center_x + pos_actual)
    pygame.draw.circle(screen, (0, 0, 255), (actual_px, center_y + 10), 6)
    # 추정 위치 (빨간색 원)
    est_px = int(center_x + pos_est)
    pygame.draw.circle(screen, (255, 0, 0), (est_px, center_y - 10), 6)
    # (옵션) 측정값 위치 (회색 원)
    # meas_px = int(center_x + pos_meas)
    # pygame.draw.circle(screen, (150, 150, 150), (meas_px, center_y), 4)

    pygame.display.flip()        # 화면 업데이트 (더블 버퍼링)
    clock.tick(50)               # 50 FPS로 지연

# 종료 처리
pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
import pygame
import numpy as np

# Pygame 초기화
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D Kalman Filter Visualization")
clock = pygame.time.Clock()

# 시뮬레이션 파라미터
dt = 1.0
accel_noise_std = 0.2
measurement_noise_std = 5.0

# 상태: [x, y, vx, vy]
true_state = np.array([100.0, 100.0, 2.0, 1.5])
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
B = np.zeros((4, 2))  # 제어 없음
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])
Q = np.eye(4) * accel_noise_std**2
R = np.eye(2) * measurement_noise_std**2

# 초기 추정값
x_est = np.array([0.0, 0.0, 0.0, 0.0])
P = np.eye(4) * 1000

# 메인 루프
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 실제 시스템 업데이트
    process_noise = np.random.multivariate_normal([0, 0, 0, 0], Q)
    true_state = A @ true_state + process_noise

    # 벽 충돌 처리
    if not (0 < true_state[0] < WIDTH):
        true_state[2] *= -1
    if not (0 < true_state[1] < HEIGHT):
        true_state[3] *= -1

    # 측정
    measurement = H @ true_state + np.random.multivariate_normal([0, 0], R)

    # 예측
    x_pred = A @ x_est
    P_pred = A @ P @ A.T + Q

    # 갱신
    y = measurement - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    # 시각화
    screen.fill((255, 255, 255))
    # 실제 위치
    pygame.draw.circle(screen, (0, 0, 255), (int(true_state[0]), int(true_state[1])), 8)
    # 측정 위치
    pygame.draw.circle(screen, (180, 180, 180), (int(measurement[0]), int(measurement[1])), 6)
    # 추정 위치
    pygame.draw.circle(screen, (255, 0, 0), (int(x_est[0]), int(x_est[1])), 8)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()


In [18]:
import pygame
import numpy as np

# Pygame 초기화
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D Kalman Filter with Random Acceleration")
clock = pygame.time.Clock()

# 시뮬레이션 파라미터
dt = 1.0
accel_noise_std = 0.2
measurement_noise_std = 5.0

# 상태: [x, y, vx, vy]
true_state = np.array([100.0, 100.0, 2.0, 1.5])
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
B = np.array([[0.5 * dt**2, 0],
              [0, 0.5 * dt**2],
              [dt, 0],
              [0, dt]])
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])
Q = np.eye(4) * accel_noise_std**2
R = np.eye(2) * measurement_noise_std**2

# 초기 추정값
x_est = np.array([0.0, 0.0, 0.0, 0.0])
P = np.eye(4) * 1000

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 임의의 가속도 생성
    ax = np.random.uniform(-0.1, 0.1)
    ay = np.random.uniform(-0.1, 0.1)
    u = np.array([ax, ay])

    # 실제 시스템 업데이트 (가속도 포함)
    process_noise = np.random.multivariate_normal([0, 0, 0, 0], Q)
    true_state = A @ true_state + B @ u + process_noise

    # 벽 충돌 처리
    if not (0 < true_state[0] < WIDTH):
        true_state[2] *= -1
    if not (0 < true_state[1] < HEIGHT):
        true_state[3] *= -1

    # 측정
    measurement = H @ true_state + np.random.multivariate_normal([0, 0], R)

    # 예측
    x_pred = A @ x_est + B @ u
    P_pred = A @ P @ A.T + Q

    # 갱신
    y = measurement - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    # 시각화
    screen.fill((255, 255, 255))
    # 실제 위치
    pygame.draw.circle(screen, (0, 0, 255), (int(true_state[0]), int(true_state[1])), 8)
    # 측정 위치
    pygame.draw.circle(screen, (180, 180, 180), (int(measurement[0]), int(measurement[1])), 6)
    # 추정 위치
    pygame.draw.circle(screen, (255, 0, 0), (int(x_est[0]), int(x_est[1])), 8)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()


In [17]:
import pygame
import numpy as np

# Pygame 초기화
pygame.init()
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("2D Kalman Filter with Accurate Collision Response")
clock = pygame.time.Clock()

# 시뮬레이션 파라미터
dt = 1.0
accel_noise_std = 0.2
measurement_noise_std = 5.0

# 상태: [x, y, vx, vy]
true_state = np.array([100.0, 100.0, 2.0, 1.5])
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
B = np.array([[0.5 * dt**2, 0],
              [0, 0.5 * dt**2],
              [dt, 0],
              [0, dt]])
H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0]])
Q = np.eye(4) * accel_noise_std**2
R = np.eye(2) * measurement_noise_std**2

# 초기 추정값
x_est = np.array([0.0, 0.0, 0.0, 0.0])
P = np.eye(4) * 1000

# 정확한 반사 적용 함수
def reflect(state, u, bounds, dt):
    pos = state[:2]
    vel = state[2:]
    acc = u

    collided = False
    t_hit = 1.0
    hit_axis = -1  # 0 for x, 1 for y

    for i in range(2):  # x, y 차원
        p0 = pos[i]
        v = vel[i]
        a = acc[i]
        bound_min = 0
        bound_max = bounds[i]

        for bound in [bound_min, bound_max]:
            c = p0 - bound
            b = v
            a_ = 0.5 * a

            # 2차 방정식 해
            discriminant = b**2 - 4 * a_ * c
            if abs(a_) > 1e-6 and discriminant >= 0:
                sqrt_d = np.sqrt(discriminant)
                for t_candidate in [(-b - sqrt_d) / (2 * a_), (-b + sqrt_d) / (2 * a_)]:
                    if 0 <= t_candidate <= dt and t_candidate < t_hit:
                        t_hit = t_candidate
                        hit_axis = i
                        collided = True
            elif abs(a_) <= 1e-6 and abs(b) > 1e-6:
                t_candidate = -c / b
                if 0 <= t_candidate <= dt and t_candidate < t_hit:
                    t_hit = t_candidate
                    hit_axis = i
                    collided = True

    if not collided:
        # 충돌 없으면 전체 이동
        return A @ state + B @ u
    else:
        # 충돌 시점까지 이동
        A1 = np.array([[1, 0, t_hit, 0],
                       [0, 1, 0, t_hit],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])
        B1 = np.array([[0.5 * t_hit**2, 0],
                       [0, 0.5 * t_hit**2],
                       [t_hit, 0],
                       [0, t_hit]])
        state = A1 @ state + B1 @ u

        # 속도 반사
        state[2 + hit_axis] *= -1

        # 남은 시간 동안 이동
        t_remain = dt - t_hit
        A2 = np.array([[1, 0, t_remain, 0],
                       [0, 1, 0, t_remain],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]])
        B2 = np.array([[0.5 * t_remain**2, 0],
                       [0, 0.5 * t_remain**2],
                       [t_remain, 0],
                       [0, t_remain]])
        state = A2 @ state + B2 @ u

        # 위치 클램핑
        # state[0] = np.clip(state[0], 0, bounds[0])
        # state[1] = np.clip(state[1], 0, bounds[1])

        return state

# 메인 루프
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 무작위 가속도
    u = np.random.uniform(-0.1, 0.1, size=2)

    # 실제 상태 업데이트 (충돌 감지 및 반사 포함)
    true_state = reflect(true_state, u, (WIDTH, HEIGHT), dt)

    # 측정 생성
    measurement = H @ true_state + np.random.multivariate_normal([0, 0], R)

    # 칼만 필터 예측 및 보정
    x_pred = A @ x_est + B @ u
    P_pred = A @ P @ A.T + Q
    y = measurement - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    # 시각화
    screen.fill((255, 255, 255))
    pygame.draw.circle(screen, (0, 0, 255), (int(true_state[0]), int(true_state[1])), 8)     # 파란: 실제
    pygame.draw.circle(screen, (180, 180, 180), (int(measurement[0]), int(measurement[1])), 6)  # 회색: 측정
    pygame.draw.circle(screen, (255, 0, 0), (int(x_est[0]), int(x_est[1])), 8)              # 빨간: 추정

    pygame.display.flip()
    clock.tick(30)

pygame.quit()
